In [1]:
import getpass
import os


def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:··········


In [2]:
pip install -U --quiet langgraph "langchain[openai]" langchain-community langchain-text-splitters faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field
from typing import Literal
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json

with open('/content/drive/MyDrive/NLP-11/data/text_descriptions.json') as f:
  data = json.load(f)


In [5]:
from langchain.schema import Document
doc_splits = [Document(page_content=doc["description"]) for doc in data]

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
            model_name='nlpai-lab/KoE5',
            model_kwargs={"device": "cuda"},
            encode_kwargs={"normalize_embeddings": True, "batch_size": 32},
)

<ipython-input-6-ddb187b323ba>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/17.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [8]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents=doc_splits, embedding=embedding_model,
)


In [9]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_financial_research_report",
    "Search and return information about Finance",
)

In [10]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder


model = HuggingFaceCrossEncoder(model_name='Dongjin-kr/ko-reranker')
compressor = CrossEncoderReranker(model=model, top_n=10)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vectorstore.as_retriever(search_kwargs={"k": 100}))



config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

In [11]:
retriever_tool_re = create_retriever_tool(
    compression_retriever,
    "retrieve_financial_research_report",
    "Search and return information about Finance",
)

In [12]:
response_model = init_chat_model("openai:gpt-4o-mini", temperature=0)


def generate_query_or_respond(state: MessagesState):
    """Call the model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """
    response = (
        response_model
        .bind_tools([retriever_tool_re]).invoke(state["messages"])
    )
    return {"messages": [response]}

In [40]:
# Revised: Single 100-doc retrieve, staged rerank using retry_count with optional query rewrite

from typing import Literal
from pydantic import BaseModel, Field
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage

# === Embedding 기반 FAISS retriever 세팅 ===
retriever_100 = vectorstore.as_retriever(search_kwargs={"k": 100})

# === CrossEncoder 기반 reranker ===
reranker_model = HuggingFaceCrossEncoder(model_name="Dongjin-kr/ko-reranker")
compressor = CrossEncoderReranker(model=reranker_model, top_n=10)

# === LLM ===
response_model = init_chat_model("openai:gpt-4o-mini", temperature=0)
grade_model = init_chat_model("openai:gpt-4o-mini", temperature=0)

# === Nodes ===
def rewrite_question(state: dict):
    question = state["messages"][0].content if hasattr(state["messages"][0], 'content') else state["messages"][0]["content"]
    prompt = f"""
    Look at the input and try to reason about the underlying semantic intent.
    Here is the initial question:
     -------
    {question}
     -------
    Formulate an improved question:
    """
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [HumanMessage(content=response.content)]}

def retrieve_once(state: dict):
    if not state.get("messages"):
        raise ValueError("State does not contain any messages.")
    query = state["messages"][0].content if hasattr(state["messages"][0], 'content') else state["messages"][0]["content"]
    docs = retriever_100.get_relevant_documents(query)
    return {
        "messages": state["messages"],
        "retrieved_docs": docs,
        "retry_count": 0
    }

def rerank_stage(state: dict):
    query = state["messages"][0].content if hasattr(state["messages"][0], 'content') else state["messages"][0]["content"]
    retry = state["retry_count"]
    start, end = retry * 30, (retry + 1) * 30
    docs = state["retrieved_docs"][start:end]
    state["retry_count"] += 1
    reranked = compressor.compress_documents(docs, query=query)
    content = "\n\n".join([d.page_content for d in reranked])
    state["messages"].append({"role": "tool", "content": content})
    return state

class GradeDocuments(BaseModel):
    binary_score: str = Field(description="'yes' if relevant, 'no' if not")

def grade_documents(state: dict) -> Literal["generate_answer", "rerank_stage"]:
    GRADE_PROMPT = (
        "You are a grader assessing relevance of a retrieved document to a user question.\n"
        "Here is the retrieved document: \n\n{context}\n\n"
        "Here is the user question: {question}\n"
        "Even if the document contains keyword(s) or semantic meaning related to the user question, You have to grade 'is it really relevant?'.\n"
        "Give a binary score 'yes' or 'no' to indicate whether the document is relevant to the question."
    )
    question = state["messages"][0].content if hasattr(state["messages"][0], 'content') else state["messages"][0]["content"]
    context = state["messages"][-1]["content"]
    prompt = GRADE_PROMPT.format(question=question, context=context)
    result = grade_model.with_structured_output(GradeDocuments).invoke([
        {"role": "user", "content": prompt}
    ])
    score = result.binary_score

    if score == "yes" or state["retry_count"] >= 3:
        return "generate_answer"
    else:
        state["retry_count"] += 1
        return "rerank_stage"

def generate_answer(state: dict):
    question = state["messages"][0].content if hasattr(state["messages"][0], 'content') else state["messages"][0]["content"]
    context = state["messages"][-1]["content"]
    prompt = f"""
    You are an assistant for question-answering tasks in the financial industry.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise.
    Question: {question}
    Context: {context}
    """
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [HumanMessage(content=response.content)]}

# === LangGraph 구성 ===
workflow = StateGraph(dict)
workflow.add_node("rewrite_question", rewrite_question)
workflow.add_node("retrieve_once", retrieve_once)
workflow.add_node("rerank_stage", rerank_stage)
workflow.add_node("generate_answer", generate_answer)

workflow.add_edge(START, "rewrite_question")
workflow.add_edge("rewrite_question", "retrieve_once")
workflow.add_edge("retrieve_once", "rerank_stage")
workflow.add_conditional_edges("rerank_stage", grade_documents, {
    "generate_answer": "generate_answer",
    "rerank_stage": "rerank_stage"
})
workflow.add_edge("generate_answer", END)

graph = workflow.compile()


In [41]:
from langchain_core.messages import HumanMessage

for chunk in graph.stream(
    {
        "messages": [
            HumanMessage(content="CJ제일제당의 2025년 매출액은 얼마로 예상돼?")
        ]
    }
):
    for node, update in chunk.items():
        print("Update from node", node)
        print(update["messages"][-1])
        print("\n\n")

Update from node rewrite_question
content='CJ제일제당의 2025년 매출액 예상치에 대한 분석이나 예측 자료가 있나요?' additional_kwargs={} response_metadata={}



Update from node retrieve_once
content='CJ제일제당의 2025년 매출액 예상치에 대한 분석이나 예측 자료가 있나요?' additional_kwargs={} response_metadata={}



Update from node rerank_stage
{'role': 'tool', 'content': '자료: QuantiWise, 신한투자증권 주: 2024, 2025, 2026년은 컨센서스 기준(CJ제일제당은 자체 추정)\n\nCJ제일제당의 24년 3분기 실적은 연결기준 매출액 7.5조원(+1.3% YoY), 영업 이익 4,206억원(+6.2% YoY, OPM 5.6%)으로 시장기대치를 하회할 전망이다. 1)국내식품은 내수 소비 부진 영향으로 가공식품 및 소재식품 외형이 축소되며 원가 부담 완화에도 수익성 하락을 예상한다. 2)해외식품은 K-Food 인기 속 미 주 지역 중심 성장을 전망한다. 미국은 냉동 식품 시장 축소 및 경쟁 심화에도 동사의 피자와 만두 등 주력 제품 점유율 상승으로 달러기준 5~6% 매출 성장 을 시현 중인 것으로 파악된다. 또한 아태/유럽 지역은 중국과 일본이 부진한 반 면 유럽과 호주 지역은 메인스트림 입점 확대로 매출이 고성장 중이다. 3)바이오 는 판가 하락을 물량 성장으로 방어했으나 전분기 대비 매출 및 이익 부진이 불 가피할 전망이며 4)F&C는 사료 가격 정상화 및 베트남과 인도네시아의 계절적 수요 부진으로 전분기 대비 이익이 축소됐으나 흑자 기조를 유지할 전망이다.\n\nCJ대한통운을 제외한 CJ제일제당의 3Q24 매출액과 영업이익은 각각 4조 6,204억원 (-1.1% YoY), 2,764억원 (+0.4% YoY)을 시현했다. 견조한 해외 가공식품 흐름 및 